In [5]:
%load_ext sql
%sql sqlite:///store.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Задание 1
Вывести 5 самых больших по размеру (в байтах) треков жанра Blues и формата MPEG. Только название и размер.


In [13]:
%%sql
SELECT t.Name, t.Bytes
  FROM tracks t
JOIN genres g, media_types m
WHERE t.GenreId = g.GenreId
  and g.Name = "Blues"
  and t.MediaTypeId = m.MediaTypeId
  and m.Name LIKE 'MPEG%'
ORDER
  BY t.Bytes DESC
LIMIT 5

 * sqlite:///store.db
Done.


Name,Bytes
Old Love,15780747
Running On Faith,12536275
Lonely Stranger,10894406
Layla,9490542
Tears In Heaven,9032835


## Задание 2
Вывести названия всех треков и их исполнителей для всех треков жанра Rock, приобретенных сотрудниками Oracle.


In [14]:
%%sql
SELECT t.Name, a.Name as Author
  FROM tracks t
JOIN artists a,
     Customers c,
     genres g,
     invoice_items ii,
     invoices i,
     albums al

WHERE t.AlbumId = al.AlbumId
  and al.ArtistId = a.ArtistId
  and t.TrackId = ii.TrackId
  and ii.InvoiceId = i.InvoiceId
  and i.CustomerId = c.CustomerId
  and t.GenreId = g.GenreId
  and c.Company = "Oracle"
  and g.Name = "Rock"

 * sqlite:///store.db
Done.


Name,Author
Sunday Bloody Sunday,U2
New Year's Day,U2
That's The Way,Led Zeppelin
Ten Years Gone,Led Zeppelin
Achilles Last Stand,Led Zeppelin
Tea For One,Led Zeppelin
No Quarter,Led Zeppelin
Bad Moon Rising,Creedence Clearwater Revival
Down On The Corner,Creedence Clearwater Revival
Up Around The Bend,Creedence Clearwater Revival


## Задание 3
Для каждой компании вывести общее количество купленных джазовых композиций (если 0, можно не выводить).


In [15]:
%%sql
SELECT c.Company, sum(ii.Quantity) as "Количество купленных композиций"
  FROM tracks t
JOIN genres g, invoice_items ii, invoices i, Customers c
WHERE g.Name = "Jazz"
  and t.GenreId = g.GenreId
  and t.TrackId = ii.TrackId
  and ii.InvoiceId = i.InvoiceId
  and i.CustomerId = c.CustomerId
GROUP
  BY c.Company
  HAVING c.Company not NULL

 * sqlite:///store.db
Done.


Company,Количество купленных композиций
Apple Inc.,3
BMW,1
Google Inc.,16
JetBrains s.r.o.,3
Microsoft Corporation,19
Oracle,3
SAP,2
Telus,2


## Задание 4
Вывести топ 3 плейлиста по продолжительности (продолжительность плейлиста = сумма длин треков в нём). Только название и продолжительность (в часах).


In [18]:
%%sql
SELECT p.Name, sum(t.Milliseconds)/3600000 as "duration in hours"
  FROM playlists p
JOIN playlist_track pt, tracks t
WHERE t.TrackId = pt.TrackId
  and pt.PlaylistId = p.PlaylistId
GROUP
  BY p.PlaylistId
ORDER BY sum(t.Milliseconds) DESC
LIMIT 3

 * sqlite:///store.db
Done.


Name,duration in hours
From That TV Show,138
Hidden Gems,138
90’s Music,10


## Задание 5
Для каждого набора (жанр, тип медиа) вывести количество треков в них, причем вывести только те наборы, для которых **все** треки стоят больше 1$ и для которых в наборе есть хотя бы один трек.

In [19]:
%%sql
SELECT g.Name as Genre, m.Name as MediaType, count(t.Name) as Count
FROM tracks t
JOIN genres g, media_types m
WHERE t.GenreId = g.GenreId
  and t.MediaTypeId = m.MediaTypeId
GROUP
  BY g.Name, m.Name
    HAVING count(t.TrackId) > 0 and min(t.UnitPrice) >= 1

 * sqlite:///store.db
Done.


Genre,MediaType,Count
Blues,AAC audio file,9
Comedy,MPEG audio file,17
Sci Fi & Fantasy,MPEG audio file,26
Science Fiction,MPEG audio file,13
TV Shows,MPEG audio file,93
